Testing various AIs with various lookup-table "value" functions on tic-tac-toe

In [1]:
import numpy as np 

Hashes out all possible board trajectories, then grades them.
Working backwards from all finished states (with either 0, 0.5, or 1 grade), for any non-finished
state, assigns maximum value of all possible future moves if my turn, minimum value if
opponent's turn.

The function assumes that it's "symbol"'s turn.

Speed: O(n!) where n is # of slots -> ~O(3^n) with dynamic programming to make a full lookup table

Simplicity: idk, took ~an hour for me to code up

In [11]:
utilities = {}
def utility(board: np.array, symbol, opp_symbol) -> int:
    #print(board)
    init_score = grade_board(board)
    if init_score != -1:
        return init_score
    
    all_traj = []
    for row in range(3):
        for col in range(3):
            if board[row][col] == 0:
                board_copy = np.copy(board)
                board_copy[row][col] = symbol
                all_traj.append(board_copy)
    
    grades = []
    for traj in all_traj:
        key = (traj.tobytes(), opp_symbol)
        if not (key in utilities.keys()):
            utilities[key] = utility(traj, opp_symbol, symbol)
        grades.append(utilities[key])
    # grades = [utility(traj, opp_symbol, symbol) for traj in all_traj]
    #print(all_traj)
    #print(grades)
    if symbol in grades:
        return symbol
    if 0.5 in grades:
        return 0.5
    return opp_symbol

"""
Grades a board -- either the winner, 0.5 if draw, or -1 if not finished
(given one player is 1, the other is 2)
Inspiration: https://gist.github.com/qianguigui1104/edb3b11b33c78e5894aad7908c773353 
"""
def grade_board(board: np.array):
    for row in range(3):
        if board[row][0] == board[row][1] == board[row][2] and board[row][0] != 0:
            return board[row][0]
    for col in range(3):
        if board[0][col] == board[1][col] == board[2][col] and board[0][col] != 0:
            return board[0][col]
        
    if board[0][0] == board[1][1] == board[2][2] and board[0][0] != 0:
        return board[0][0]
    if board[0][2] == board[1][1] == board[2][0] and board[0][2] != 0:
        return board[0][2]
    
    full = True
    for row in range(3):
        for col in range(3):
            if board[row][col] == 0:
                full = False
    if full:
        return 0.5
    return -1

def hash(board, symbol) -> int:
    out = 0
    primes = [2, 3, 5, 7, 11, 13, 17, 19, 23]
    for row in range(3):
        for col in range(3):
            out += primes[3*row + col] * board[row][col]
    out += 29 * symbol
    return

print(utility(np.zeros((3, 3)), 1, 2)) #expected: 0.5
print(len(utilities))
print(utility(np.array([[1, 0, 1], [2, 2, 0], [1, 0, 2]]), 1, 2)) 
#X _ X | O O _ | X _ O; expected: 1
print(utility(np.array([[1, 0, 0], [0, 2, 0], [0, 0, 0]]), 1, 2))
#X _ _ | _ O _ | _ _ _; expected: 0.5

0.5
5477
1
0.5


Is this a utility function model or a value model? The recursion part might make it a value model,
but we're also effectively using brute-force here (the only "heuristic" here is end-game score).

What would a reward/value lookup table for tic-tac-toe look like? 
- If there's two-in-a-row of our symbol and the other spot is blank, max reward (we can win on the next turn)
- If there's two-in-a-row of their symbol and the other spot is blank, all moves besides filling that blank spot have min reward
- If we have the opportunity to create a "fork" (a setup where we have two symbols in two different row/column/diagonals, making it impossible for the opponent to stop us next turn from winning), max reward
- Otherwise, maybe having zero opp_symbol and >= 1 symbol in a row/column/diagonal is positive reward?
    - Maybe having a symbol in a corner or the center is good
    - These don't seem very meaningful/indicative of anything, and are harder to code up

e.g. see https://github.com/SamarpanCoder2002/Tic-Tac-Toe-AI/blob/main/Tic-Tac-Toe-AI.py for a heuristics AI (~430 lines!)

Heuristics: see https://blog.ostermiller.org/tic-tac-toe-strategy/ for an example

Notable points:
- Value/Reward looks simliar to utility in returning a number, but doesn't do recursion
- In return, grade_board has to be more complicated (act() is simpler than utility())

In [16]:
#Value/Reward -- O(n) * (runtime of grade_board) where n = # cells
def act(board, symbol, opp_symbol) -> tuple:
    init_score = grade_board(board, symbol, opp_symbol)
    if init_score == 0 or init_score == 1: #game is over
        return init_score
    
    all_traj = []
    moves = []
    for row in range(3):
        for col in range(3):
            if board[row][col] == 0:
                board_copy = np.copy(board)
                board_copy[row][col] = symbol
                all_traj.append(board_copy)
                moves.append((row, col))
    
    grades = [grade_board(traj, opp_symbol, symbol) for traj in all_traj]
    #grades contains reward/value for each possible move
    if symbol in grades:
        return symbol
    return moves[grades.index(min(grades))] #tuple resulting in the worst position for the opponent

#Calculates reward/value
def grade_board(board: np.array, symbol, opp_symbol):
    """
    Pseudocode:
    for all one move trajectories, resulting in new_board:
        assign rewards to notable differences (e.g. if move resulted in two-in-a-row)
        sum rewards
    return sum
    """
    return np.random.rand()


print(act(np.zeros((3, 3)), 1, 2))

(1, 2)


Claim: there exists a structure w/n NN (e.g. some Python function) that can be described as one of these non-degenerately (where non-degenerate <-> predictive OOD)

ex: chess
- Utility: {all possible set of moves} -> {0, 0.5, 1}
- Reward: -3 pts if lose knight, etc.
- Value: rating the “total value” of the current board (e.g. by summing up all the values of the pieces; this would produce a pretty bad chess AI, but not that bad)
- Heuristics: e.g. if Sicilian, play 5… a6

In this case, the code becomes more "complicated", harder to write and harder to interpret as we sacrifice more speed. If we change the game a little bit (e.g. make it so that each player can place 1s or 2s; the first player to get three-in-a-row of either symbol wins), the heuristics will also fail to generalize, whereas a small modification to the grade_board and/or utility function will result in the utility function working. (You do have to carefully modify grade_board, though; I guess in some sense, we use the heuristic that we are in a 3x3 board, only 1s and 2s are played, etc.)

In [ ]:
#Heuristics
def act_heuristics(board: np.array, symbol, opp_symbol) -> tuple:
    """
    Example pseudocode:
    if (\exists two symbols in a line) and (third space is blank):
        return third space
    if (\exists space with one symbol and one blank in two lines sharing that space):
        return space  #can create a fork
    (so on for all heuristics) ...
    return ...
    """
    return np.random.rand(board.shape)

In [1]:
### ChatGPT's code below, not mine
### Modified by me in places

from itertools import product

def init_board():
    """ Initialize an empty Tic-Tac-Toe board. """
    return [[' ' for _ in range(3)] for _ in range(3)]

def print_board(board):
    """ Print the Tic-Tac-Toe board. """
    for row in board:
        print('|'.join(row))
        print('-' * 5)

def get_possible_moves(board):
    """ Get a list of possible moves for the current board state. """
    moves = []
    for x, y in product(range(3), range(3)):
        if board[x][y] == ' ':
            moves.append((x, y))
    return moves

def check_winner(board):
    """ Check if there is a winner or if the game is a draw. """
    # Check rows, columns, and diagonals for a win
    for i in range(3):
        if board[i][0] == board[i][1] == board[i][2] != ' ':
            return board[i][0]
        if board[0][i] == board[1][i] == board[2][i] != ' ':
            return board[0][i]
    if board[0][0] == board[1][1] == board[2][2] != ' ' or board[0][2] == board[1][1] == board[2][0] != ' ':
        return board[1][1]

    # Check for draw
    if all(board[x][y] != ' ' for x, y in product(range(3), range(3))):
        return 'Draw'

    return None

# Test the functions
board = init_board()
board[0][0] = 'X'
board[1][1] = 'O'
board[2][2] = 'X'
print_board(board)
print("Possible Moves: ", get_possible_moves(board))
print("Winner: ", check_winner(board))

X| | 
-----
 |O| 
-----
 | |X
-----
Possible Moves:  [(0, 1), (0, 2), (1, 0), (1, 2), (2, 0), (2, 1)]
Winner:  None


In [2]:
def minimax(board, depth, is_maximizing, alpha, beta):
    """ Minimax algorithm with Alpha-Beta Pruning. """
    winner = check_winner(board)
    if winner == 'X':
        return 10 - depth
    elif winner == 'O':
        return depth - 10
    elif winner == 'Draw':
        return 0

    if is_maximizing:
        best_score = -float('inf')
        for move in get_possible_moves(board):
            board[move[0]][move[1]] = 'X'
            score = minimax(board, depth + 1, False, alpha, beta)
            board[move[0]][move[1]] = ' '
            best_score = max(best_score, score)
            alpha = max(alpha, score)
            if beta <= alpha:
                break
        return best_score
    else:
        best_score = float('inf')
        for move in get_possible_moves(board):
            board[move[0]][move[1]] = 'O'
            score = minimax(board, depth + 1, True, alpha, beta)
            board[move[0]][move[1]] = ' '
            best_score = min(best_score, score)
            beta = min(beta, score)
            if beta <= alpha:
                break
        return best_score

def find_best_move(board, player):
    """ Find the best move for the current player. """
    best_move = None
    if player == 'X':
        best_score = -float('inf')
        for move in get_possible_moves(board):
            board[move[0]][move[1]] = 'X'
            score = minimax(board, 0, False, -float('inf'), float('inf'))
            board[move[0]][move[1]] = ' '
            if score > best_score:
                best_score = score
                best_move = move
    else:
        best_score = float('inf')
        for move in get_possible_moves(board):
            board[move[0]][move[1]] = 'O'
            score = minimax(board, 0, True, -float('inf'), float('inf'))
            board[move[0]][move[1]] = ' '
            if score < best_score:
                best_score = score
                best_move = move
    return best_move

# Test the Minimax algorithm
board = init_board()
board[0][0] = 'X'
board[1][1] = 'O'
print_board(board)
best_move = find_best_move(board, 'X')
print("Best Move for 'X':", best_move)

X| | 
-----
 |O| 
-----
 | | 
-----
Best Move for 'X': (0, 1)


In [5]:
from itertools import product

def board_to_string(board):
    """ Convert board to a string representation. """
    return ''.join(''.join(row) for row in board)

def string_to_board(board_str):
    """ Convert string representation back to board. """
    return [list(board_str[i:i+3]) for i in range(0, len(board_str), 3)]

def generate_all_possible_boards():
    """ Generate all possible boards (valid and invalid). """
    return [''.join(p) for p in product(' XO', repeat=9)]

def is_valid_board(board_str):
    """ Check if a board is valid. """
    x_count = board_str.count('X')
    o_count = board_str.count('O')
    return x_count == o_count or x_count == o_count + 1

def generate_lookup_table(find_best_move_alg):
    """ Generate the lookup table for all valid game states. """
    lookup_table = {}
    for board_str in generate_all_possible_boards():
        if is_valid_board(board_str):
            board = string_to_board(board_str)
            winner = check_winner(board)
            if winner is None:  # Only calculate moves if the game is not already over
                player = 'X' if board_str.count('X') == board_str.count('O') else 'O'
                best_move = find_best_move_alg(board, player)
                lookup_table[board_str] = best_move
    return lookup_table

# Generate the lookup table (this may take some time)
lookup_table = generate_lookup_table(find_best_move)

# Test the lookup table with an example board
test_board = "X O   OX "
print("Best Move for the board:", test_board, "is", lookup_table.get(test_board))

Best Move for the board: X O   OX  is (1, 1)


ChatGPT:

"To create a Tic-Tac-Toe AI that uses a reward function and heuristics to determine the best move without relying on the Minimax algorithm, we need to define a set of rules or heuristics that evaluate the desirability of each move based on the current board state. This approach will analyze each move individually and assign a reward or score based on its potential to lead to a win, block the opponent, or contribute to a strategic advantage.

Here are some potential heuristics we can use:

- Winning Move: If a move creates a line of three 'X's or 'O's, it gets the highest score.
- Block Opponent's Win: If a move blocks the opponent from winning on their next turn, it gets a high score.
- Create Opportunity: Moves that create two potential winning opportunities in the next turn (e.g., two lines of two 'X's or 'O's with a blank space) are scored favorably.
- Center Control: Taking the center square, if available, is usually advantageous.
- Corner Control: Occupying corners can be strategic as they are part of two potential winning lines.
- Edge Control: Less preferable than corners or center, but still useful.
- Randomness: In case of multiple moves having the same score, a random move can be selected to add unpredictability.

Each possible move will be evaluated based on these heuristics, and the one with the highest score will be selected."

In [18]:
import random

def heuristic_score(board, move, player):
    """ Calculate the heuristic score for a given move. """
    score = 0
    x, y = move
    opponent = 'O' if player == 'X' else 'X'

    # Create a temporary board for scoring
    temp_board = [row[:] for row in board]
    temp_board[x][y] = player

    # Winning move
    if check_winner(temp_board) == player:
        score += 100

    # Block opponent's win
    temp_board[x][y] = opponent
    if check_winner(temp_board) == opponent:
        score += 90
    temp_board[x][y] = player

    # Create opportunities
    for dx, dy in product([-1, 0, 1], repeat=2):
        if 0 <= x + dx < 3 and 0 <= y + dy < 3 and temp_board[x + dx][y + dy] == player:
            score += 10

    # Center control
    if move == (1, 1):
        score += 50

    # Corner control
    if move in [(0, 0), (0, 2), (2, 0), (2, 2)]:
        score += 30

    # Edge control
    if move in [(0, 1), (1, 0), (1, 2), (2, 1)]:
        score += 20

    return score

def find_best_move_value(board, player):
    """ Find the best move based on rewards based on heuristics. """
    best_score = -1
    best_move = None
    for move in get_possible_moves(board):
        score = heuristic_score(board, move, player)
        if score > best_score or (score == best_score and random.random() > 0.5):
            best_score = score
            best_move = move
    return best_move

# Test the heuristic-based AI
board = init_board()
board[0][0] = 'X'
board[1][1] = 'O'
print_board(board)
best_move = find_best_move_value(board, 'X')
print("Best Heuristic Move for 'X':", best_move)


X| | 
-----
 |O| 
-----
 | | 
-----
Best Heuristic Move for 'X': (2, 0)


In [19]:
# Can the bots spot forks?
board = init_board()
board[0][0] = 'X'
board[1][1] = 'O'
board[0][2] = 'O'
board[2][2] = 'X' 
print_board(board)
# Optimal move is (2, 0)
best_move_util = find_best_move(board, 'X')
best_move_heur = find_best_move_value(board, 'X')
print(best_move_util, best_move_heur)
print(board_to_string(board))

X| |O
-----
 |O| 
-----
 | |X
-----
(2, 0) (2, 0)
X O O   X


In [20]:
#Lookup table for reward
lookup_table_value = generate_lookup_table(find_best_move_value)
print(lookup_table_value)

{'         ': (1, 1), '        X': (1, 1), '       X ': (1, 1), '       XO': (1, 1), '       OX': (1, 1), '      X  ': (1, 1), '      X O': (1, 1), '      XXO': (1, 1), '      XO ': (1, 1), '      XOX': (1, 1), '      O X': (1, 1), '      OX ': (1, 1), '      OXX': (1, 1), '     X   ': (1, 1), '     X  O': (1, 1), '     X XO': (1, 1), '     X O ': (1, 1), '     X OX': (0, 2), '     XX O': (1, 1), '     XXO ': (1, 1), '     XXOO': (1, 1), '     XO  ': (1, 1), '     XO X': (0, 2), '     XOX ': (1, 1), '     XOXO': (1, 1), '     XOOX': (0, 2), '     O  X': (1, 1), '     O X ': (1, 1), '     O XX': (2, 0), '     OX  ': (1, 1), '     OX X': (2, 1), '     OXX ': (2, 2), '     OXXO': (0, 2), '     OXOX': (1, 1), '     OOXX': (1, 1), '    X    ': (2, 0), '    X   O': (2, 0), '    X  XO': (0, 1), '    X  O ': (0, 2), '    X  OX': (0, 0), '    X X O': (0, 2), '    X XO ': (0, 2), '    X XOO': (0, 2), '    X O  ': (2, 2), '    X O X': (0, 0), '    X OX ': (0, 1), '    X OXO': (0, 1), '    X OOX':

In [24]:
# Run some games between the utility and value functions
NUM_GAMES = 100
results = [0, 0, 0]
for i in range(NUM_GAMES):
    board = init_board()
    counter = i % 2 # set so that both the utility and value tables get to play first NUM_GAMES / 2 times each
    
    while check_winner(board) == None:
        move = (0,)
        board_str = board_to_string(board)
        curr_player = 'X' if board_str.count('X') == board_str.count('O') else 'O'
        if counter % 2 == 0: # alternating between utility, value
            # move = lookup_table.get(board_str)
            move = find_best_move(board, curr_player)
        else:
            # move = lookup_table_value.get(board_str)
            move = find_best_move_value(board, curr_player)
        board[move[0]][move[1]] = curr_player
        counter += 1
    
    winner = check_winner(board)
    if (winner == 'X' and i % 2 == 0) or (winner == 'O' and i % 2 == 1):
        results[0] += 1
    elif winner == 'Draw':
        results[2] += 1
    else:
        results[1] += 1
        
print(results)

[0, 0, 100]


ChatGPT:
"After running 10,000 games where both players used the heuristic-based AI, the results are as follows:
- Player 'X' won 1,421 games.
- Player 'O' did not win any games.
- The remaining 8,579 games ended in a draw.

These results indicate that the heuristic-based AI tends to lead to a high number of draw games, especially in a game like Tic-Tac-Toe where optimal play from both sides often results in a draw. The advantage of going first (Player 'X') seems to have translated into more wins compared to Player 'O', who did not secure any victories. This outcome could be influenced by the specific heuristics used and their effectiveness in different game situations."

In [25]:
### Using only heuristics

def find_optimal_move(board, player):
    """ Find the optimal move based on Tic-Tac-Toe strategy heuristics. """
    opponent = 'O' if player == 'X' else 'X'
    empty = ' '
    moves = get_possible_moves(board)

    # Define strategies in order of priority
    strategies = [
        ('Win', lambda b, p: p),  # Win
        ('Block', lambda b, p: opponent),  # Block opponent's win
        ('Fork', lambda b, p: p),  # Create a fork
        ('Block Fork', lambda b, p: opponent),  # Block opponent's fork
        ('Center', lambda b, p: p if b[1][1] == empty else None),  # Play center
        ('Opposite Corner', lambda b, p: p),  # Play opposite corner
        ('Empty Corner', lambda b, p: p),  # Play an empty corner
        ('Empty Side', lambda b, p: p)  # Play an empty side
    ]

    for strategy_name, strategy_player in strategies:
        for move in moves:
            temp_board = [row[:] for row in board]
            temp_board[move[0]][move[1]] = strategy_player(temp_board, player)
            if check_winner(temp_board) == player:
                return move

    # If no strategic move found, choose a random move
    return random.choice(moves) if moves else None

# Generate the lookup table for the heuristic-based optimal AI
def generate_optimal_lookup_table():
    all_possible_boards = [''.join(p) for p in product(' XO', repeat=9)]
    optimal_lookup_table = {}

    for board_str in all_possible_boards:
        if is_valid_board(board_str):
            board = string_to_board(board_str)
            player = 'X' if board_str.count('X') == board_str.count('O') else 'O'
            best_move = find_optimal_move(board, player)
            optimal_lookup_table[board_str] = best_move

    return optimal_lookup_table

optimal_lookup_table = generate_optimal_lookup_table()

# Sample test to verify the table
sample_board = "X O   OX "
print("Optimal Move for the board:", sample_board, "is", optimal_lookup_table.get(sample_board))

Optimal Move for the board: X O   OX  is (1, 1)


In [26]:
def play_game(ai1, ai2):
    board = init_board()
    current_player = 'X'
    while True:
        move = ai1(board, current_player) if current_player == 'X' else ai2(board, current_player)
        if move is not None:
            board[move[0]][move[1]] = current_player
        winner = check_winner(board)
        if winner is not None:
            return winner
        current_player = 'O' if current_player == 'X' else 'X'

def run_games_between_ais(ai1, ai2, n):
    """ Run n games between two AIs and count the results. """
    results = {'X': 0, 'O': 0, 'Draw': 0}
    for _ in range(n):
        winner = play_game(ai1, ai2)
        results[winner] += 1
    return results

# Define the three AIs: heuristic-based, heuristic optimal, and minimax
ai_heuristic = find_best_move_value
ai_heuristic_optimal = find_optimal_move
ai_minimax = lambda board, player: optimal_lookup_table[board_to_string(board)]

# Run 10,000 games for each pair of AIs
n_games = 10000
results_heuristic_vs_optimal = run_games_between_ais(ai_heuristic, ai_heuristic_optimal, n_games)
results_heuristic_vs_minimax = run_games_between_ais(ai_heuristic, ai_minimax, n_games)
results_optimal_vs_minimax = run_games_between_ais(ai_heuristic_optimal, ai_minimax, n_games)

# Combine the results
total_results = {
    'Heuristic vs Optimal': results_heuristic_vs_optimal,
    'Heuristic vs Minimax': results_heuristic_vs_minimax,
    'Optimal vs Minimax': results_optimal_vs_minimax
}

print(total_results)

{'Heuristic vs Optimal': {'X': 9746, 'O': 0, 'Draw': 254}, 'Heuristic vs Minimax': {'X': 10000, 'O': 0, 'Draw': 0}, 'Optimal vs Minimax': {'X': 7100, 'O': 2407, 'Draw': 493}}


It seems like 'X' has an advantage (not surprising) and minimax did poorly (surprising). Note that each of the three pairings above are run with one side always using 'X' and the other always using 'O'.

In [27]:
def run_swapped_games_between_ais(ai1, ai2, n):
    """ Run n games between two AIs, swapping who starts as 'X' and 'O' each game. """
    results = {'X': 0, 'O': 0, 'Draw': 0}
    for i in range(n):
        if i % 2 == 0:
            winner = play_game(ai1, ai2)
        else:
            winner = play_game(ai2, ai1)
        
        # Adjust results based on who was 'X' in each game
        if winner == 'Draw':
            results['Draw'] += 1
        elif (winner == 'X' and i % 2 == 0) or (winner == 'O' and i % 2 != 0):
            results['X'] += 1
        else:
            results['O'] += 1
    return results

# Run 1,000 games for each pair of AIs, swapping sides
n_swapped_games = 1000
results_heuristic_vs_optimal_swapped = run_swapped_games_between_ais(ai_heuristic, ai_heuristic_optimal, n_swapped_games)
results_heuristic_vs_minimax_swapped = run_swapped_games_between_ais(ai_heuristic, ai_minimax, n_swapped_games)
results_optimal_vs_minimax_swapped = run_swapped_games_between_ais(ai_heuristic_optimal, ai_minimax, n_swapped_games)

# Combine the results
total_swapped_results = {
    'Heuristic vs Optimal (Swapped)': results_heuristic_vs_optimal_swapped,
    'Heuristic vs Minimax (Swapped)': results_heuristic_vs_minimax_swapped,
    'Optimal vs Minimax (Swapped)': results_optimal_vs_minimax_swapped
}

print(total_swapped_results)

{'Heuristic vs Optimal (Swapped)': {'X': 903, 'O': 28, 'Draw': 69}, 'Heuristic vs Minimax (Swapped)': {'X': 1000, 'O': 0, 'Draw': 0}, 'Optimal vs Minimax (Swapped)': {'X': 512, 'O': 460, 'Draw': 28}}


Ok something might be off here if minimax is losing